In [51]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


# Context

In [52]:
pd.set_option('display.max_rows', 20)
sample = pd.read_csv('data/sample_submission.csv')
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
display(train)
# pd.set_option('display.max_rows', None)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,2,"Bell, Adele",female,62.00,0,0,PC 15008,14.86,D17243,C
99996,99996,0,2,"Brown, Herman",male,66.00,0,0,13273,11.15,NaN,S
99997,99997,0,3,"Childress, Charles",male,37.00,0,0,NaN,9.95,NaN,S
99998,99998,0,3,"Caughlin, Thomas",male,51.00,0,1,458654,30.92,NaN,S


# Data quality assessment

In [56]:
assert all(train['PassengerId'] == np.arange(10 ** 5)), 'PassengerId must be in the range from 0 to 100000 inclusive'
assert all(train['Survived'].apply(lambda survived: survived in [0, 1])), 'Survived must have a value equal to 0 or 1'
assert all(train['Pclass'].apply(lambda pclass: pclass in range(1, 4))), \
'Pclass must have a value in the range from 1 to 3 inclusive'
assert all(train['Name'].str.contains('[A-Z]{1}[a-z]*, [A-Z]{1}[a-z]*', regex=True)), 'The Name must match the template'
assert all(train['Sex'].apply(lambda gender: gender in ['male', 'female'])), 'The Sex must be equal to either male or female'
assert all(train['Age'].astype(str).str.contains('\d\d?.\d\d?|nan', regex=True)), \
'Age must match the presented template and is equal to NaN'
assert all(train['SibSp'].apply(lambda sibsp: sibsp in range(9))), 'SubSp is not in the acceptable range'
assert all(train['Parch'].apply(lambda parch: parch in range(10))), 'Parch is not in the acceptable range'
assert all(train['Ticket'].astype(str).str.contains(
    '\d{4,5}|[A-Z].[A-Z5].?|[A-Z4]{2}.?|A/[4,5]|SOTON/O.Q.|W./C.|SC/PARIS|C|nan', regex=True)), \
'''Ticket must match one of the templates:
4 digits, 5 digits, letter.letter, 2 letters, A.5., A4., A/4, A/5, SOTON/O.Q., W./C., SC/PARIS, C'''
assert all(train['Cabin'].astype(str).str.contains('[A-GT]\d{4,5}|nan', regex=True)), \
'Cabin must match one of the templates: letter from A to G or T and 4 or 5 digits'
assert all(train['Embarked'].astype(str).apply(lambda embarked: embarked in ['C', 'Q', 'S', 'nan'])), \
'Embarked must be equal to C, Q, S or NaN'
train['PassengerId'] = train['PassengerId'].dropna().astype(np.int64)
train['Survived'] = train['Survived'].dropna().astype(np.int64)
train['Pclass'] = train['Pclass'].dropna().astype(np.int64)
train['Name'] = train['Name'].dropna().astype(str)
train['Sex'] = train['Sex'].dropna().astype(str)
train['Age'] = train['Age'].dropna().astype(np.float64)
train['SibSp'] = train['SibSp'].dropna().astype(np.int64)
train['Parch'] = train['Parch'].dropna().astype(np.int64)
train['Ticket'] = train['Ticket'].dropna().astype(str)
train['Fare'] = train['Fare'].dropna().astype(np.float64)
train['Cabin'] = train['Cabin'].dropna().astype(str)
train['Embarked'] = train['Embarked'].dropna().astype(str)
print('The testing was completed successfully')

The testing was completed successfully


# Data exploration

In [240]:
df = train['PassengerId'] == np.arange(10 ** 5)
display(df)

0        True
1        True
2        True
3        True
4        True
         ... 
99995    True
99996    True
99997    True
99998    True
99999    True
Name: PassengerId, Length: 100000, dtype: bool

# Summary